In [1]:
from collections import Counter
from itertools import combinations, chain

class Tile:
    def __init__(self, suit, unit):
        self.suit = suit
        self.unit = unit

    def __lt__(self, other):
        return (self.suit, self.unit) <  (other.suit, other.unit)
    
    def __eq__(self, other):
        return (self.suit, self.unit) == (other.suit, other.unit)

    def __hash__(self):
        return hash((self.suit, self.unit))
    
    def __str__(self):
        return f"{self.suit}{self.unit}"

def Group_Sets(hand):
    freq = Counter(hand)
    pair = [[x] * 2 for x in freq if freq[x] > 1]
    pong = [[x] * 3 for x in freq if freq[x] > 2]
    chow = []
    for card in freq: #for card in freq
        if isinstance(card.unit, str):
            continue

        seqn = (
            Tile(card.suit, card.unit),
            Tile(card.suit, card.unit + 1),
            Tile(card.suit, card.unit + 2)
        )
        
        chow += [seqn] * min(freq[x] for x in seqn)
    
    yield pair; yield pong; yield chow

def decompose_meld(hand):
    pair, pong, chow = Group_Sets(hand)

    handfreq = Counter(hand)
    meldfreq = Counter({item: 0 for item in hand})

    #tallies all meld DCMPs
    numcount = len(hand) // 3
    while numcount >= 0:
        for testcase in combinations(pong + chow, numcount):
            testfreq = Counter(chain(*testcase))
            if any(testfreq[x] > handfreq[x] for x in testfreq):
                continue
            meldfreq.update(testfreq)
            numcount = 0
        numcount = numcount - 1

    for card in meldfreq:
        meldfreq[card] = round(meldfreq[card] / handfreq[card], 2)
    maxcount = max(*meldfreq.values())

    parser = []
    for card in meldfreq:
        temp = meldfreq[card]

        for _ in range(handfreq[card]):
            if meldfreq[card] >= maxcount:
                break

            meldfreq[card] = meldfreq[card] + temp
            #yield card
            parser.append(card)

    if len(parser) < 3:
        #if parser.ispair():
            #winner!
        iswaiting = True
    return parser if parser else []

arr = [
    Tile("b", 4), 
    Tile("b", 4),
    Tile("b", 4),
    Tile("b", 5),
    Tile("b", 5),
    Tile("b", 5),
    Tile("b", 6),
    Tile("b", 6),
    Tile("b", 6),
    Tile("b", 7),
    Tile("b", 7),
    Tile("b", 7),
    Tile("b", 8),
    Tile("b", 8),
    Tile("b", 8),
    Tile("b", 9),
    Tile("b", 9)
]

arr = [
    Tile("b", 2),
    Tile("b", 3),
    Tile("b", 4),
    Tile("b", 5),
    Tile("b", 6),
    Tile("b", 7),
    Tile("b", 8),
]

print(*[str(x) for x in decompose_meld(arr)])

b2 b5 b8


In [106]:
import random
from collections import Counter

tiles = [
    "b1", "b2", "b3", "b4", "b5", "b6", "b7", "b8", "b9",
    "s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8", "s9",
    "c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c9",
    "tN", "tS", "tW", "tE", "tG", "tR", "tW", "fR", "fB"
] * 4


def tiles_needed(hand, freq):
    handfreq = Counter(hand)
    
    handneed = Counter()
    cardneed = {item: Counter() for item in hand}
    #needfreq = hand's need freq
    #cardneed = card's need freq
    #remove cards with max hand's need when card is removed from hand
    needtiles = {}

    for card in handfreq:
        if card[1].isalpha():
            completing_seqn = {
                (card, card) : (card, card)
            }

        else:
            seqn = [
                card[0] + str(int(card[1]) - 1),
                card[0] + str(int(card[1]) + 0),
                card[0] + str(int(card[1]) + 1),
                card[0] + str(int(card[1]) + 2)
            ]

            completing_seqn = {
                (seqn[1], seqn[1]) : (seqn[1], seqn[1]),
                (seqn[1], seqn[2]) : (seqn[0], seqn[3]),
                (seqn[1], seqn[3]) : (seqn[2], )
            }
        
        for pre_meld, cmp_meld in completing_seqn.items():
            if (Counter(pre_meld) - handfreq):
                continue

            handneed.update(cmp_meld)
            for card in pre_meld:
                cardneed[card].update(cmp_meld)
    
    needcount = {}
    for card in handfreq:
        needtiles[card] = (handneed - cardneed[card]).keys()
        needcount[card] = sum(freq[i] for i in needtiles[card]) # available needtiles

    maxcount = 0
    if needcount:
        maxcount = max(needcount.values())
    return [x for x in needcount if needcount[x] == maxcount]

def tiles_needed2(hand, freq):
    handfreq = Counter(hand)
    
    handneed = []
    cardneed = {item: 0 for item in hand}

    for card in handfreq:
        if card[1].isalpha():
            completing_seqn = {
                (card, card) : (card, card)
            }

        else:
            seqn = [
                card[0] + str(int(card[1]) - 1),
                card[0] + str(int(card[1]) + 0),
                card[0] + str(int(card[1]) + 1),
                card[0] + str(int(card[1]) + 2)
            ]

            completing_seqn = {
                (seqn[1], seqn[1]) : (seqn[1], seqn[1]),
                (seqn[1], seqn[2]) : (seqn[0], seqn[3]),
                (seqn[1], seqn[3]) : (seqn[2], )
            }
        
        for pre_meld, cmp_meld in completing_seqn.items():
            if (Counter(pre_meld) - handfreq):
                continue

            handneed.extend(cmp_meld)
            for card in pre_meld:
                cardneed[card] += sum(freq[i] for i in cmp_meld) / handfreq[card]

    maxcount = 0
    if cardneed:
        maxcount = min(cardneed.values())
    return [x for x in cardneed if cardneed[x] == maxcount]

#hand = ["s5", "s7", "s8"]

"""hands = [
    ["b3", "b6", "b7", "b8", "b8", "b8", "c1", "c5", "c8", "s4", "s7", "s9"],
    ["b4", "b5", "b7", "c1", "c3", "fR", "s2", "s3", "s8", "tE", "tG", "tW"],
    ["b1", "b2", "b2", "fR", "s2", "s5", "s8", "s9", "tG", "tR", "tS", "tW"],
    ["b1", "b7", "c1", "c3", "c4", "s3", "s3", "s4", "s5", "s5", "tN", "tW"],
    ["b1", "b3", "b6", "b6", "c4", "c6", "c8", "s3", "s5", "tR", "tS", "tS"],
    ["b2", "b2", "b5", "b5", "c9", "fB", "s1", "s1", "s3", "s9", "tE", "tR"],
    ["b4", "b5", "b6", "b7", "b8", "c2", "c4", "fB", "s3", "s4", "tE", "tG"],
    ["b2", "b4", "b5", "c1", "c1", "c9", "s5", "s6", "s7", "s9", "tN", "tW"],
    ["b3", "b4", "c1", "c2", "c5", "c6", "s3", "s4", "s6", "s7", "s8", "tG"],
    ["b1", "b4", "b8", "c5", "c9", "fB", "s1", "s2", "s6", "tN", "tR", "tR"],
    ["b4", "b7", "b7", "b9", "c5", "c6", "c6", "c7", "c8", "s5", "tN", "tR"],
    ["b1", "b4", "c1", "c2", "c4", "c6", "c9", "fB", "s3", "s9", "s9", "tS"],
    ["c1", "c2", "c3", "c4", "c5", "c6", "c7", "c9", "s4", "s7", "tN", "tW"],
    ["b1", "b1", "b2", "b3", "c2", "c3", "c6", "c7", "s3", "s8", "tG", "tW"],
    ["b1", "b5", "b6", "b6", "b7", "b8", "c1", "c3", "c4", "fB", "s1", "tE"],
    ["b1", "b1", "c5", "c6", "c7", "c8", "s1", "s2", "s4", "s6", "tE", "tS"],
    ["b1", "b2", "b2", "b4", "b6", "c7", "fB", "s3", "s5", "s8", "s9", "tS"],
    ["b1", "b3", "b9", "c3", "c3", "c8", "fR", "s2", "s6", "s9", "s9", "tE"],
    ["b2", "b2", "b3", "b6", "c1", "c7", "c8", "s1", "s2", "s9", "tR", "tW"],
    ["b3", "b3", "b7", "b9", "c1", "c6", "c7", "c7", "fR", "fR", "tG", "tR"],
    ["b1", "b5", "c3", "c6", "c7", "c8", "c9", "s2", "s8", "tN", "tR", "tW"],
    ["b2", "b4", "c6", "c8", "s1", "s2", "s5", "s6", "s6", "tW", "tW", "tW"],
    ["b2", "b6", "c2", "c4", "c7", "c8", "fR", "s1", "s1", "s5", "s6", "tW"],
    ["b5", "b8", "b8", "b8", "b9", "c6", "c7", "c9", "s1", "s4", "tN", "tW"],
    ["b1", "b6", "c1", "s1", "s2", "s5", "s8", "s8", "s9", "tN", "tS", "tW"],
    ["b5", "b5", "b6", "b8", "c3", "c6", "fR", "s6", "s7", "s8", "tN", "tS"],
    ["b7", "b9", "c7", "c8", "c8", "fR", "s2", "s5", "s7", "tE", "tS", "tW"],
    ["b1", "b5", "b9", "b9", "c8", "c8", "s2", "s2", "s3", "s5", "s9", "tR"],
    ["b4", "b5", "b5", "b6", "b9", "c3", "c8", "s1", "s4", "s5", "s7", "tW"],
    ["b1", "b2", "b9", "c4", "c7", "c9", "s5", "s6", "s7", "tE", "tR", "tR"],
    ["b2", "b6", "b9", "c2", "c3", "c6", "c7", "c8", "c8", "s7", "s8", "tR"],
    ["b4", "b6", "c1", "c2", "c5", "c8", "s3", "s6", "s9", "tG", "tN", "tR"],
    ["b6", "b9", "b9", "c2", "c4", "c5", "c6", "s2", "s7", "tS", "tS", "tW"],
    ["b2", "b4", "b5", "c1", "c7", "s1", "s1", "s3", "tE", "tG", "tG", "tS"],
    ["b2", "b5", "b9", "c1", "c8", "fB", "fR", "s6", "tE", "tR", "tS", "tW"],
    ["b3", "b5", "b6", "s3", "s4", "s4", "s5", "s7", "s8", "s8", "tG", "tR"],
    ["b3", "b7", "b9", "c2", "c3", "fR", "s2", "s4", "tN", "tR", "tS", "tW"],
    ["b9", "c1", "c9", "fR", "s1", "s3", "s3", "s4", "s7", "s8", "tR", "tR"],
    ["b1", "b7", "b8", "b9", "c2", "c7", "c8", "s1", "s4", "s6", "s7", "tR"],
    ["b2", "b2", "b5", "b6", "b7", "b8", "c2", "c7", "s4", "s4", "s5", "tW"],
    ["b1", "b2", "b3", "b5", "b7", "c3", "c6", "c8", "s6", "s8", "tS", "tW"],
    ["b1", "b1", "b8", "b8", "c2", "c2", "c2", "s6", "tE", "tN", "tR", "tW"],
    ["b3", "b6", "c3", "c6", "s2", "s7", "s9", "s9", "tG", "tN", "tN", "tW"],
    ["b5", "b6", "c2", "c4", "c5", "c8", "s1", "s3", "s3", "s5", "tG", "tR"],
    ["b7", "b8", "c5", "c9", "fB", "s7", "s7", "tE", "tE", "tG", "tW", "tW"],
    ["b2", "b2", "b3", "b4", "b6", "b8", "b9", "b9", "c7", "tR", "tS", "tW"],
    ["b4", "c1", "c2", "c5", "fR", "s1", "s3", "s4", "tE", "tG", "tR", "tR"],
    ["b3", "c3", "c4", "c5", "c6", "c8", "c8", "s1", "s1", "s4", "s9", "tW"],
    ["b1", "b4", "b5", "c8", "c8", "c9", "fB", "s1", "s3", "s5", "s8", "tG"],
    ["b3", "b4", "b4", "b5", "b6", "b9", "b9", "c4", "fR", "tG", "tN", "tN"],
    ["b6", "c3", "fR", "fR", "s1", "s2", "s3", "s4", "s4", "s5", "s5", "s9"],
    ["b4", "b9", "c2", "c2", "c7", "fB", "fR", "s1", "s2", "s7", "s7", "s8"],
    ["b1", "b3", "b5", "c2", "c3", "c6", "c7", "c8", "s2", "s7", "s9", "s9"],
    ["b1", "b5", "b9", "c1", "c3", "c3", "c4", "s1", "tE", "tE", "tS", "tW"],
    ["b1", "b2", "b7", "b8", "b9", "c4", "c7", "c9", "fB", "s2", "s5", "s9"],
    ["b3", "b7", "c1", "c1", "c5", "c8", "s6", "s7", "s7", "tN", "tW", "tW"],
    ["b4", "b6", "b9", "c2", "c2", "fB", "s2", "s2", "s3", "s8", "s8", "tG"],
    ["b4", "c2", "c4", "c6", "c6", "c7", "s1", "s7", "s8", "tG", "tR", "tS"],
    ["b5", "b5", "b8", "c1", "c5", "fB", "s1", "s3", "s8", "tE", "tN", "tS"],
    ["b3", "c1", "c3", "c6", "c8", "c8", "s6", "s6", "s6", "tN", "tW", "tW"],
    ["b1", "b5", "b9", "c2", "c4", "c5", "c6", "s2", "s3", "s3", "tE", "tE"],
    ["b4", "b4", "b7", "b8", "s2", "s5", "s7", "tN", "tN", "tW", "tW", "tW"],
    ["b2", "b3", "b7", "c2", "c2", "c5", "s1", "s2", "s2", "s6", "s9", "tR"],
    ["c9", "fB", "s1", "s1", "s2", "s2", "s7", "s7", "tN", "tN", "tR", "tW"],
    ["b4", "b5", "c3", "c8", "fB", "s3", "s6", "s6", "tE", "tR", "tW", "tW"],
    ["b8", "b8", "b9", "c2", "c3", "s1", "s4", "s8", "s8", "s9", "tG", "tR"],
    ["b5", "b7", "b7", "b8", "c1", "c2", "c9", "fR", "s3", "s4", "s7", "tE"],
    ["b6", "b9", "c3", "c4", "c9", "fB", "fR", "s4", "s6", "tG", "tS", "tW"],
    ["b2", "b3", "b4", "b7", "b9", "c6", "c6", "s1", "s6", "s9", "tN", "tS"],
    ["b3", "c2", "c4", "c4", "c5", "fB", "s1", "s7", "tE", "tG", "tG", "tS"],
    ["c7", "c8", "fB", "s1", "s4", "s6", "s7", "s7", "tN", "tR", "tW", "tW"],
    ["b3", "b4", "b8", "b9", "c7", "s2", "s3", "s6", "tN", "tR", "tR", "tW"],
    ["b3", "c3", "c9", "fR", "s5", "s6", "s8", "s9", "tE", "tG", "tN", "tN"],
    ["b1", "b2", "c5", "s3", "s5", "s6", "s7", "s8", "s9", "tN", "tR", "tR"],
    ["b6", "b9", "c1", "c2", "c3", "c5", "c7", "s9", "tG", "tN", "tR", "tW"],
    ["b5", "b5", "b6", "b7", "b9", "c6", "fR", "s1", "s6", "tE", "tR", "tW"],
    ["b4", "c1", "c6", "s1", "s3", "s3", "s7", "s8", "s9", "tG", "tG", "tS"],
    ["b2", "b4", "b6", "b7", "b8", "fB", "s3", "s4", "s6", "s6", "s9", "tW"],
    ["b2", "c3", "c8", "fB", "fB", "s1", "s3", "s8", "s8", "tR", "tR", "tS"],
    ["b2", "b6", "b9", "c2", "c5", "c6", "c6", "fB", "fR", "s5", "s7", "s7"],
    ["b2", "b5", "b5", "b6", "c1", "c5", "c7", "c8", "fB", "s1", "s8", "s9"],
    ["b1", "b9", "c1", "c3", "c4", "c9", "fR", "s1", "s2", "s2", "s4", "s8"],
    ["b3", "b4", "b5", "c3", "c3", "c4", "c6", "fR", "s2", "tG", "tG", "tW"],
    ["b1", "b2", "b3", "b5", "c1", "c4", "c5", "s6", "tE", "tE", "tR", "tW"],
    ["b5", "b6", "b8", "c1", "c6", "c7", "fB", "s5", "tE", "tN", "tN", "tR"],
    ["b2", "b6", "b7", "c1", "c6", "c6", "c8", "c9", "fR", "s6", "s6", "s8"],
    ["b1", "b3", "b4", "b5", "b9", "c4", "c6", "c8", "fR", "tN", "tS", "tW"],
    ["b3", "b8", "c3", "c9", "fR", "s1", "s1", "s4", "s4", "s6", "tE", "tW"],
    ["b2", "b3", "b9", "b9", "c1", "c4", "fR", "s2", "s5", "s8", "s8", "tW"],
    ["b4", "b6", "b6", "c4", "c6", "c9", "s3", "s8", "s9", "tR", "tR", "tS"],
    ["b1", "b5", "c1", "c2", "c6", "c7", "c9", "fB", "s1", "tG", "tR", "tW"],
    ["b3", "b5", "b9", "c3", "c4", "fB", "s5", "s8", "s9", "s9", "tW", "tW"],
    ["b5", "b8", "c3", "c4", "c5", "fB", "fB", "s1", "s2", "tG", "tG", "tW"],
    ["b1", "b1", "b3", "b5", "b6", "c6", "c8", "fR", "s4", "tS", "tW", "tW"],
    ["b2", "b2", "b2", "b4", "b7", "b9", "c7", "c9", "s2", "s5", "s7", "tW"],
    ["b2", "b8", "c1", "c3", "c8", "s2", "s5", "s8", "s9", "tR", "tW", "tW"],
    ["b6", "c3", "c5", "c5", "c9", "s1", "s4", "s4", "s9", "tE", "tR", "tS"],
    ["b4", "b6", "b8", "b9", "c1", "c2", "c6", "fR", "s5", "s8", "tN", "tW"],
    ["b9", "c1", "c2", "c3", "c7", "c8", "c8", "fR", "s4", "s5", "s9", "s9"],
    ["b3", "b5", "c2", "c4", "fB", "s1", "s1", "s4", "tE", "tR", "tS", "tW"]
]    
for hand in hands:
    freq = Counter(tiles)
    for tile in hand:
        freq[tile] -= 1

    hand = sorted(hand, key=lambda item: (item[0], item[1]))
    result = sorted(tiles_needed2(hand, freq), key=lambda item: (item[0], item[1]))
    print(" ".join(result) + " ")
"""

hand = ["c2", "c2", "c2", "c3", "c3", "c4", "c5", "c6", "c6", "c7"]
tiles = [
    "b1", "b2", "b3", "b4", "b5", "b6", "b7", "b8", "b9",
    "s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8", "s9",
    "c1", "c5", "c6", "c7", "c8", "c9",
    "tN", "tS", "tW", "tE", "tG", "tR", "tW", "fR", "fB"
] * 4 + ["c2", "c2", "c2", "c3", "c3", "c4", "c4", "c4"]

freq = Counter(tiles)
print(*tiles_needed2(hand, freq))
"""hand = []
test_1 = []
test_2 = []
counter = 1000
while test_1 == test_2:
    random.shuffle(tiles)
    hand = sorted(tiles[:4], key=lambda x: (x[0], x[1]))
    freq = Counter(tiles[4:])

    test_1 = tiles_needed(hand, freq)
    test_2 = tiles_needed2(hand, freq)
    counter -= 1

    if counter < 0:
        print("empty")
        break


print(*hand)
print()
print(*tiles_needed(hand, freq))
print(*tiles_needed2(hand, freq))
"""

c2


'hand = []\ntest_1 = []\ntest_2 = []\ncounter = 1000\nwhile test_1 == test_2:\n    random.shuffle(tiles)\n    hand = sorted(tiles[:4], key=lambda x: (x[0], x[1]))\n    freq = Counter(tiles[4:])\n\n    test_1 = tiles_needed(hand, freq)\n    test_2 = tiles_needed2(hand, freq)\n    counter -= 1\n\n    if counter < 0:\n        print("empty")\n        break\n\n\nprint(*hand)\nprint()\nprint(*tiles_needed(hand, freq))\nprint(*tiles_needed2(hand, freq))\n'

In [ ]:
def near_cards(hand, freq):
    nearcount = {}
    for card in set(hand):
        if isinstance(card.unit, str):
            seqn = [card]
        else:
            seqn = [
                Tile(card.suit, card.unit - 2),
                Tile(card.suit, card.unit - 1),
                Tile(card.suit, card.unit),
                Tile(card.suit, card.unit + 1),
                Tile(card.suit, card.unit + 2)
            ]

        nearcount[card] = sum(freq[x] for x in seqn)
    
    min_count = min(nearcount.values())
    return [x for x in hand if nearcount[x] == min_count]

In [98]:
tiles = [
    "b1", "b2", "b3", "b4", "b5", "b6", "b7", "b8", "b9",
    "s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8", "s9",
    "c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c9",
    "tN", "tS", "tW", "tE", "tG", "tR", "tW", "fR", "fB"
] * 4

import random
for i in range(100):
    random.seed((i + 100) ** (3/2))
    random.shuffle(tiles)
    hand = tiles[:12]

    print("[" + ", ".join([f'"{item}"' for item in sorted(hand)]) + "],")
    #print("new List<string>() {" + ", ".join([f'"{item}"' for item in hand]) + "},")


["b3", "b6", "b7", "b8", "b8", "b8", "c1", "c5", "c8", "s4", "s7", "s9"],
["b4", "b5", "b7", "c1", "c3", "fR", "s2", "s3", "s8", "tE", "tG", "tW"],
["b1", "b2", "b2", "fR", "s2", "s5", "s8", "s9", "tG", "tR", "tS", "tW"],
["b1", "b7", "c1", "c3", "c4", "s3", "s3", "s4", "s5", "s5", "tN", "tW"],
["b1", "b3", "b6", "b6", "c4", "c6", "c8", "s3", "s5", "tR", "tS", "tS"],
["b2", "b2", "b5", "b5", "c9", "fB", "s1", "s1", "s3", "s9", "tE", "tR"],
["b4", "b5", "b6", "b7", "b8", "c2", "c4", "fB", "s3", "s4", "tE", "tG"],
["b2", "b4", "b5", "c1", "c1", "c9", "s5", "s6", "s7", "s9", "tN", "tW"],
["b3", "b4", "c1", "c2", "c5", "c6", "s3", "s4", "s6", "s7", "s8", "tG"],
["b1", "b4", "b8", "c5", "c9", "fB", "s1", "s2", "s6", "tN", "tR", "tR"],
["b4", "b7", "b7", "b9", "c5", "c6", "c6", "c7", "c8", "s5", "tN", "tR"],
["b1", "b4", "c1", "c2", "c4", "c6", "c9", "fB", "s3", "s9", "s9", "tS"],
["c1", "c2", "c3", "c4", "c5", "c6", "c7", "c9", "s4", "s7", "tN", "tW"],
["b1", "b1", "b2", "b3", "c2", "c3", "